# Módulo 3 - Ejercicio Evaluación Final

## Fase 1 - Exploración

In [ ]:
# -----------------------------------------------------------------------
# IMPORTS
# -----------------------------------------------------------------------
# Tratamiento de datos
# -------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# -------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# -------------------------------------------------
from scipy.stats import shapiro, kstest

# Configuración
# -------------------------------------------------
pd.set_option('display.max_columns', None)      # Para visualizar todas las columnas de los DataFrames.
pd.set_option('display.max_colwidth', None)     # Para ver todo el contenido de las columnas.

# Gestión de los warnings
# -------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# Importaciones funciones propias
# -------------------------------------------------
from src import soporte_eda as sp_eda
from src import soporte_correlacion as sp_corr

### Carga de datos archivo Customer Flight Activity.csv:

In [ ]:
# Este archivo contiene información sobre la actividad de vuelo de los clientes, 
# incluyendo el número de vuelos reservados, la distancia volada, puntos acumulados y redimidos, y costos asociados a los puntos redimidos.

df_fa = pd.read_csv("files/Customer Flight Activity.csv")
df_fa.head(3)

In [ ]:
sp_eda.exploracion_basica(df_fa)

Comprobamos esas filas duplicadas:

In [ ]:
duplicados = df_fa[df_fa.duplicated(keep=False)]
duplicados.head(10)

Como se puede observar, esas filas son idénticas entre ellas (incluyendo sobre todo el id Loyalty Number y Year/Month, que son las que servirían para diferenciar unas con otras) así que podemos eliminarlas todas:

In [ ]:
df_fa = df_fa.drop_duplicates()     # Mantiene la primera y elimina las subsecuentes (keep='first' por defecto).

In [ ]:
sp_eda.exploracion_basica(df_fa, secciones=['info', 'duplicados'])

In [ ]:
sp_eda.exploracion_basica(df_fa, secciones=['num_desc'])

In [ ]:
df_fa.columns

In [ ]:
# Loyalty Number: Este atributo representa un identificador único para cada cliente dentro del programa de lealtad de la aerolínea. 
# Cada número de lealtad corresponde a un cliente específico.

In [ ]:
sp_eda.exploracion_num(df_fa,'Year', graficos=False, mostrar_estadisticas=False, mostrar_outliers=False)
# Year: Indica el año en el cual se registraron las actividades de vuelo para el cliente.

In [ ]:
sp_eda.exploracion_num(df_fa,'Month', graficos=False, mostrar_estadisticas=False, mostrar_outliers=False)
# Month: Representa el mes del año (de 1 a 12) en el cual ocurrieron las actividades de vuelo.

In [ ]:
df_fa.sample(3)

In [ ]:
sp_eda.exploracion_num(df_fa,'Flights Booked', graficos=True, mostrar_estadisticas=True, mostrar_outliers=False)
# Flights Booked: Número total de vuelos reservados por el cliente en ese mes específico.

In [ ]:
round(df_fa['Flights Booked'].value_counts(normalize=True).sort_index() * 100,2)     # Porcentajes de aparición de cada valor.

In [ ]:
sp_eda.exploracion_num(df_fa,'Flights Booked', graficos=False, mostrar_estadisticas=False, mostrar_outliers=True)

In [ ]:
sp_eda.exploracion_num(df_fa,'Flights with Companions', graficos=True, mostrar_estadisticas=True, mostrar_outliers=True)
# Flights with Companions: Número de vuelos reservados en los cuales el cliente viajó con acompañantes.

In [ ]:
round(df_fa['Flights with Companions'].value_counts(normalize=True).sort_index() * 100,2)     # Porcentajes de aparición de cada valor.

In [ ]:
sp_eda.exploracion_num(df_fa,'Total Flights', graficos=True, mostrar_estadisticas=True, mostrar_outliers=True)
# Total Flights: El número total de vuelos que el cliente ha realizado, que puede incluir vuelos reservados en meses anteriores.

In [ ]:
sp_eda.exploracion_num(df_fa,'Distance', graficos=True, mostrar_estadisticas=True, mostrar_outliers=False)
# Distance: La distancia total (presumiblemente en millas o kilómetros) que el cliente ha volado durante el mes.

In [ ]:
sp_eda.exploracion_num(df_fa,'Points Accumulated', graficos=True, mostrar_estadisticas=True, mostrar_outliers=False)
# Points Accumulated: Puntos acumulados por el cliente en el programa de lealtad durante el mes, 
# con base en la distancia volada u otros factores.

Para comprobar si realmente hay valores con decimales diferentes de 0 y por lo tanto, decidir si tiene sentido mantener la columna de tipo float y no int:

In [ ]:
# Se filtran filas con decimales no enteros y se muestran algunas:
decimales_no_enteros = df_fa[df_fa['Points Accumulated'] % 1 != 0]['Points Accumulated'].unique()

print(sorted(decimales_no_enteros)[:20])  # muestra los primeros 20 valores con decimales


In [ ]:
df_fa[df_fa['Points Accumulated'].isin([9.72,10.8,16.2])]

Efectivamente hay valores con decimales.

In [ ]:
sp_eda.exploracion_num(df_fa,'Points Redeemed', graficos=True, mostrar_estadisticas=True, mostrar_outliers=False)
# Points Redeemed: Puntos que el cliente ha redimido en el mes, posiblemente para obtener beneficios como vuelos gratis, mejoras, etc.

En la columna Points Redeemed, el 25%, 50% y 75% percentiles son 0, lo que indica que al menos el 75% de los clientes no han canjeado puntos. Por eso el boxplot se ve como una línea en 0.

In [ ]:
sp_eda.exploracion_num(df_fa,'Dollar Cost Points Redeemed', graficos=True, mostrar_estadisticas=True, mostrar_outliers=False)
# Dollar Cost Points Redeemed: El valor en dólares de los puntos que el cliente ha redimido durante el mes.

Igual que ocurría con la columna Points Redeemed, en Dollar Cost Points Redeemed el 25%, 50% y 75% percentiles son 0, lo que indica que al menos el 75% de los clientes no han canjeado puntos. Por eso el boxplot se ve como una línea en 0.

In [ ]:
df_fa.columns

### Carga de datos archivo Customer Loyalty History.csv:

In [ ]:
# Este archivo proporciona un perfil detallado de los clientes, incluyendo su ubicación, nivel educativo, ingresos, estado civil, y 
# detalles sobre su membresía en el programa de lealtad (como el tipo de tarjeta, valor de vida del cliente, y fechas de inscripción y cancelación).

df_lh = pd.read_csv("files/Customer Loyalty History.csv")
df_lh.head(3)

In [ ]:
sp_eda.exploracion_basica(df_lh)

Primero verificamos que los Loyalty Number coincidan aproximadamente entre las dos tablas:

In [ ]:
# Número total de loyalty numbers en cada tabla
total_fa = df_fa['Loyalty Number'].nunique()
total_lh = df_lh['Loyalty Number'].nunique()

# Loyalty numbers comunes entre las dos tablas
comunes = set(df_fa['Loyalty Number']).intersection(set(df_lh['Loyalty Number']))
num_comunes = len(comunes)

print(f"Total Loyalty Numbers en df_fa: {total_fa}")
print(f"Total Loyalty Numbers en df_demo: {total_lh}")
print(f"Loyalty Numbers comunes: {num_comunes}")

# Porcentaje de coincidencia respecto a cada tabla
print(f"Porcentaje de df_fa que tiene demo: {num_comunes / total_fa * 100:.2f}%")
print(f"Porcentaje de df_demo que tiene fa: {num_comunes / total_lh * 100:.2f}%")


Se dividen las columnas numéricas y categóricas para poder analizarlas por separado:

In [ ]:
numericas = df_lh.select_dtypes(include=['number']).columns.tolist()
categoricas = df_lh.select_dtypes(exclude=['number']).columns.tolist()

print("Columnas numéricas:", numericas)
print("Columnas categóricas:", categoricas)

In [ ]:
sp_eda.exploracion_num(df_lh,'Salary', graficos=True, mostrar_estadisticas=True, mostrar_outliers=False)
# Salary: Ingreso anual estimado del cliente.

In [ ]:
negativos = df_lh[df_lh['Salary'] < 0].sort_values(by='Salary')
print(f"Número de salarios negativos: {len(negativos)}")
print(negativos[['Salary']])

In [ ]:
salario_minimo = df_lh.loc[df_lh['Salary'] >= 0, 'Salary'].min()
print(f"Salario mínimo sin negativos: {salario_minimo}")

Teniendo en cuenta que:
- El salario mínimo sin contar los negativos es 15.609,0
- Q1 = 59246 --> el 25% más bajo cobra hasta 59.246,0
- Q3 = 88517.5 --> el 25% de los sueldos se encuentran por encima de 88.517,5

Tenemos un rango de valores negativos que van de -9.081,0 a -58.486,0, que en valores absolutos serían: 9.081,0 y 58.486,0.
Esos valores absolutos encajan dentro del rango actual de salarios positivos, por lo que podría considerarse un error de insercion de los datos y transformarlos.

In [ ]:
# Convertir valores negativos a positivos (valor absoluto)
df_lh['Salary'] = df_lh['Salary'].abs()

In [ ]:
sp_eda.exploracion_num(df_lh,'Salary', graficos=True, mostrar_estadisticas=True, mostrar_outliers=True)
# Salary: Ingreso anual estimado del cliente.

In [ ]:
sp_eda.exploracion_basica(df_lh, secciones=['nulos'])

In [ ]:
print("Columnas numéricas:", numericas)

In [ ]:
sp_eda.exploracion_num(df_lh,'CLV', graficos=True, mostrar_estadisticas=True, mostrar_outliers=True)
# CLV (Customer Lifetime Value): Valor total estimado que el cliente aporta a la empresa durante toda la relación que mantiene con ella.

In [ ]:
sp_eda.exploracion_num(df_lh,'Enrollment Year', graficos=True, mostrar_estadisticas=False, mostrar_outliers=False)
# Enrollment Year: Año en que el cliente se inscribió en el programa de lealtad.

In [ ]:
round(df_lh['Enrollment Year'].value_counts(normalize=True).sort_index() * 100,2)     # Porcentajes de aparición de cada valor.

In [ ]:
numericas

In [ ]:
sp_eda.exploracion_num(df_lh,'Enrollment Month', graficos=True, mostrar_estadisticas=False, mostrar_outliers=False)
# Enrollment Month: Mes en que el cliente se inscribió en el programa de lealtad.

In [ ]:
round(df_lh['Enrollment Month'].value_counts(normalize=True).sort_index() * 100,2)     # Porcentajes de aparición de cada valor.

In [ ]:
numericas

In [ ]:
sp_eda.exploracion_num(df_lh,'Cancellation Year', graficos=True, mostrar_estadisticas=True, mostrar_outliers=False)
# Cancellation Year: Año en que el cliente canceló su membresía en el programa de lealtad, si aplica.

In [ ]:
round(df_lh['Cancellation Year'].value_counts(normalize=True).sort_index() * 100,2)     # Porcentajes de aparición de cada valor.

In [ ]:
sp_eda.exploracion_num(df_lh,'Cancellation Month', graficos=True, mostrar_estadisticas=True, mostrar_outliers=False)
# Cancellation Month: Mes en que el cliente canceló su membresía en el programa de lealtad, si aplica.

In [ ]:
round(df_lh['Cancellation Month'].value_counts(normalize=True).sort_index() * 100,2)     # Porcentajes de aparición de cada valor.

In [ ]:
categoricas

In [ ]:
sp_eda.exploracion_cat(df_lh,'Province', graficos=True, mostrar_tablas=True, top=10, detectar_raras=True)
# Province: Provincia o estado de residencia del cliente (aplicable a países con divisiones provinciales o estatales, como Canadá).

In [ ]:
sp_eda.exploracion_cat(df_lh,'City', graficos=True, mostrar_tablas=True, top=10, detectar_raras=True)
# City: Ciudad de residencia del cliente.

In [ ]:
categoricas

In [ ]:
sp_eda.exploracion_cat(df_lh,'Postal Code', graficos=True, mostrar_tablas=False, top=10, detectar_raras=False)
# Postal Code: Código postal del cliente.

In [ ]:
sp_eda.exploracion_cat(df_lh,'Gender', graficos=False, mostrar_tablas=True, top=10, detectar_raras=False)
# Gender: Género del cliente (ej. Male para masculino y Female para femenino).

In [ ]:
sp_eda.exploracion_cat(df_lh,'Education', graficos=True, mostrar_tablas=True, top=10, detectar_raras=True)
# Education: Nivel educativo alcanzado por el cliente (ej. Bachelor para licenciatura, College para estudios universitarios o técnicos, etc.).

In [ ]:
sp_eda.exploracion_cat(df_lh,'Marital Status', graficos=True, mostrar_tablas=True, top=10, detectar_raras=True)
# Marital Status: Estado civil del cliente (ej. Single para soltero, Married para casado, Divorced para divorciado, etc.).

In [ ]:
sp_eda.exploracion_cat(df_lh,'Loyalty Card', graficos=True, mostrar_tablas=True, top=10, detectar_raras=True)
# Loyalty Card: Tipo de tarjeta de lealtad que posee el cliente. Esto podría indicar distintos niveles o categorías dentro del programa de lealtad.

In [ ]:
sp_eda.exploracion_cat(df_lh,'Enrollment Type', graficos=True, mostrar_tablas=True, top=10, detectar_raras=True)
# Enrollment Type: Tipo de inscripción del cliente en el programa de lealtad (ej. Standard).

### Fase 1 - Limpieza & Unión

### Gestión de nulos 'Salary'

In [ ]:
df_lh.groupby('Education')['Salary'].describe()

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(data=df_lh, x='Education', y='Salary')
plt.title('Distribución de Salary por Education')
plt.xticks(rotation=45)
plt.show()

A destacar:
- "College" tiene 0 registros, por eso todos son NaN. No hay datos de 'Salary' para esa categoría.
- Los que tienen Doctorado tienen un salario promedio mucho más alto y más dispersión.
- Los que tienen High School o menos tienen el salario promedio más bajo.
- Los demás niveles están en medio con distintos rangos.

In [ ]:
# Filtrar filas donde Salary es NaN
salary_nulls = df_lh[df_lh['Salary'].isna()]

# Ver las categorías únicas de Education para esos NaN en Salary
education_null_salary = salary_nulls['Education'].unique()

print("Categorías en Education para filas con Salary nulo:", education_null_salary)

# Comprobar si todos los Salary nulos son de la categoría 'College'
todos_son_college = all(education_null_salary == 'College')
print("¿Todos los Salary nulos son de la categoría 'College'? :", todos_son_college)


In [ ]:
sns.barplot(data=df_lh, x='Salary', y='Education', errorbar=None);

Teniendo en cuenta que en Canadá el orden de Educación es el siguiente (de menos a más):

- High School or Below --> Educación secundaria
- College --> Formación profesional post-secundaria
- Bachelor --> Universidad (título de grado)
- Master	-->	Estudios de postgrado
- Doctor	-->	Doctorado académico

Se podrían rellenar todos los nulos de College con un valor entre la mediana de HS y Bachelor. Mediana y no media porque la mediana es más robusta ante valores extremos o outliers.

In [ ]:
# Medianas de Salary por categoría educativa
mediana_hs = df_lh.loc[df_lh['Education'] == 'High School or Below', 'Salary'].median()
mediana_bach = df_lh.loc[df_lh['Education'] == 'Bachelor', 'Salary'].median()

# Valor intermedio (media de las dos medianas)
valor_intermedio = (mediana_hs + mediana_bach) / 2

# Imputar Salary en filas donde Education == 'College' y Salary es NaN
cond = (df_lh['Education'] == 'College') & (df_lh['Salary'].isna())
df_lh.loc[cond, 'Salary'] = valor_intermedio

print(f"Valor imputado para College: {valor_intermedio:.2f}")

In [ ]:
sns.barplot(data=df_lh, x='Salary', y='Education', errorbar=None);

In [ ]:
df_lh[df_lh['Salary'] == 66937.5]

In [ ]:
sp_eda.exploracion_basica(df_lh)

### Unión de los dos df en uno completo:

In [ ]:
# Asegúrate que la columna es del mismo tipo en ambos
df_fa['Loyalty Number'] = df_fa['Loyalty Number'].astype(str)
df_lh['Loyalty Number'] = df_lh['Loyalty Number'].astype(str)

# Unión directa
df_completo = df_fa.merge(df_lh, on='Loyalty Number', how='inner') 
# inner devuelve solo las filas que tienen coincidencia en ambas tablas (intersección).
# Como ya comprobamos antes que todos los Loyalty Numbers están en ambas tablas, la intersección es el total, y por tanto un inner es suficiente.
# Además, inner es un poco más eficiente y evita que aparezcan filas con datos faltantes.

print(f"Dimensiones del dataframe unido: {df_completo.shape}")
display(df_completo.head())

In [ ]:
df_completo.to_csv('files/df_completo.csv', index=False)
# index=False para que no se guarde la columna de índices.